# Условия
Дано 2 csv-файла: один с текстом — articles.csv, второй со стоп-словами — stopwords.csv.

Необходимо в Jupyter-ноутбуке выполнить следующие пункты, используя SQLite или Pandas совместно с Python (очистку данных можно проводить также либо с помощью Python, либо используя SQL или Pandas).

## Шаг 1
Прочитать файлы и создать над ними таблицы, где структуры таблиц:

### 1. articles в виде:

 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)

### 2. stopwords в виде:

 |-- word: string (nullable = true)
До выполнения задачи изначально обработать данные:

при парсинге отбросить все символы, которые не являются латинскими буквами;
привести все слова к нижнему регистру;
удалить все стоп-слова из articles с помощью таблицы stopwords.

In [161]:
import pandas as pd
import numpy as np
import re
import sqlite3 as sql

In [162]:
# Читаем файлы и создаем таблицы с помощью pandas
articles = pd.read_csv("articles.csv", delimiter=';', names=["text"], header=None)
stopwords = pd.read_csv("stopwords.csv", delimiter=';', names=["word"], header=None)

Производим отчистку текста 

In [163]:
# функция для отчистки текста 
def clean_text(text):
    # Формируем маску с использованием регулятрного выражения, 
    # чтобы оставить только латинские буквы
    cleaned_text = re.sub('[^a-zA-Z\s]', '', text)
    # Привести текст к нижнему регистру
    cleaned_text = cleaned_text.lower()
    return cleaned_text

In [164]:
# Очистка текста статей
articles['text'] = articles['text'].apply(clean_text)

In [165]:
# создаем множество стоп-слов
stopwords_set = set(stopwords['word'])

In [166]:
# Удаляем  стоп-слова из текста
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords_set]
    return ' '.join(filtered_words)


articles['text'] = articles['text'].apply(remove_stopwords)

In [167]:
# Добавление столбца ID
articles.insert(0, 'id', np.arange(1, len(articles) + 1))

#### Создание соединения с базой данных SQLite

In [168]:
conn = sql.connect('articles.db')

In [169]:
# Создаем таблицы articles и stopwords
articles.to_sql('articles', conn, if_exists='replace', index=False)

stopwords.to_sql('stopwords', conn, if_exists='replace', index=False)

733

Проверим данные в таблице 

In [170]:
articles_from_db = pd.read_sql_query("SELECT * FROM articles", conn)
articles_from_db.head()

,id,text
0,1,bradley charles cooper born january american a...
1,2,cooper enrolled mfa program actors studio begi...
2,3,cooper found greater success romantic comedy s...
3,4,labeled sex symbol media cooper named people m...
4,5,cooper born january abington township near phi...


In [171]:
conn.close()

## Шаг 2
Извлечь коллокации в тексте articles.csv. Это комбинации слов, которые часто встречаются вместе. Например, «smart boss» или «linings playbook». Чтобы найти совпадения, нужно использовать метрику NPMI (нормализованная точечная взаимная информация).

PMI двух слов a и b определяется как:

PMI(a,b) = ln(P(a,b)/(P(a)*P(b)))

где P(a,b) — вероятность двух слов, идущих подряд, а P(a) и  P(b)  — вероятности слов a и b соответственно.
Вам нужно будет оценить вероятности встречаемости слов, то есть  P(a)  — это вероятность появления слова a, которая считается как отношение количества повторений слова a на общее количество слов, P(ab) — вероятность появления комбинации ab относительно всех пар, которая считается как отношение количества повторений пары ab на общее количество пар.
NPMI вычисляется как:

NPMI(a,b) = -( PMI(a,b)/lnP(a,b))

Это нормализует величину в диапазон [-1; 1].
Таким образом, для каждой комбинации слов ab посчитать NPMI и вывести на экран TOP-50 самых популярных коллокаций, отсортированных по убыванию значения NPMI.
Комбинацию слов ab объединить пробелом.

Пример вывода

|smart boss             |1.0044481503489615|

|smart home             |1.0041328218065497|

In [172]:
from collections import Counter
from math import log

Cоздаем функцию для вычисления NPMI

In [173]:
def npmi(p_a, p_b, p_ab):
    pmi = log(p_ab / (p_a * p_b))
    return -pmi / log(p_ab)

In [174]:
# Считаем слова и пары
word_count = Counter()
pair_count = Counter()
total_words = 0
total_pairs = 0

In [175]:
conn = sql.connect('articles.db')
cur = conn.cursor()
cur.execute("SELECT text FROM articles")

In [176]:
for row in cur.fetchall():
    words = row[0].split()
    total_words += len(words)
    total_pairs += len(words) - 1
    word_count.update(words)
    pair_count.update(zip(words[:-1], words[1:]))

In [177]:
# Вычисляем вероятность для слов и пар слов
word_probs = {word: count / total_words for word, count in word_count.items()}
pair_probs = {pair: count / total_pairs for pair, count in pair_count.items()}

In [178]:
# Вычисляем NPMI для каждой пары слов
npmi_values = {pair: npmi(word_probs[pair[0]], word_probs[pair[1]], pair_prob) for pair, pair_prob in pair_probs.items()}

In [179]:
# Создаем таблицу NPMI 
npmi_table = pd.DataFrame([(pair[0], pair[1], npmi) for pair, npmi in npmi_values.items()], columns=['word_a', 'word_b', 'npmi'])


In [180]:
# Сортируем значения и оставляем ТОП 50
top_50_npmi = npmi_table.sort_values(by='npmi', ascending=False)[:50]

In [181]:
from prettytable import PrettyTable

# изменяем представление 

top_50_npmi['words'] = top_50_npmi.apply(lambda row: row[0] + ' ' + row[1], axis=1)
top_50_npmi = top_50_npmi.drop(['word_a', 'word_b'], axis=1)
table = PrettyTable(['Words', 'NPMI'])
for row in top_50_npmi.itertuples(index=False):
    words = row[1]
    npmi = row[0]
    table.add_row([words, npmi])

In [183]:
# Выводим резлультат
print(table)

+------------------------------------+--------------------+
|               Words                |        NPMI        |
+------------------------------------+--------------------+
|              fish fry              | 1.0052239644191399 |
|          nightmare alley           | 1.0051069538755188 |
|          linings playbook          | 1.0051069538755188 |
|            los angeles             | 1.0051069538755188 |
|          guardians galaxy          | 1.0049706878472697 |
|            willy wanker            | 1.004805384273404  |
|         iberian peninsula          | 1.004805384273404  |
|           licorice pizza           | 1.004805384273404  |
|            barack obama            | 1.004805384273404  |
|          ella fitzgerald           | 1.0045902343009177 |
|         severely deformed          | 1.0045902343009177 |
|           clint eastwood           | 1.0045902343009177 |
|             angkor wat             | 1.0045902343009177 |
|           milvian bridge           | 1